In [16]:
%load_ext autoreload
%autoreload 2
from charting import create_performance_chart
from utils import get_llm_answers, get_llm_stats, load_all_llm_answers_from_json, model_clean
from auto_eval import (
    create_all_llm_eval_messages, 
    extract_all_scores, 
    create_auto_eval_json, 
    get_llm_eval_responses, 
    score_multiple_choice_answers,
    validation_func,
    extract_valid_json,
)
from multiple_choice import construct_multiple_choice_question
from hotz_reflection import construct_hotz_reflection_question

from datetime import datetime
import pandas as pd
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Variables

In [ ]:
answer_models = [
    ("gpt-4-turbo-preview", "litellm"),
    # ("gpt-4o", "litellm"),
    # ("gpt-4o-mini-2024-07-18", "litellm"),
    # ("o1-preview", "custom"),
    # ("bedrock/meta.llama3-70b-instruct-v1:0", "litellm"),
    # ("Meta-Llama-3-1-70B-Instruct-ostu.eastus.models.ai.azure.com", "custom"),
    # ("Meta-Llama-3-1-405B-Instruct-jjo.eastus.models.ai.azure.com", "custom"),
    # ("mistral-large-latest", "custom"),
    # ("mistral/open-mixtral-8x22b", "litellm"),
    # ("claude-3-opus-20240229", "litellm"),
    # ("gemini-1.5-pro", "custom"),
    # ("gemini-1.5-pro-exp-0801", "custom"),
    # ("command-r-plus", "litellm"),
    # ("claude-3-5-sonnet-20240620", "litellm"),
]

answer_rounds = 1 # Number of rounds of questions to ask each model
answer_hyperparams = {
    "batch_size": 5,  # Max number of questions to send to a model at once (10 is sensible)
    "temperature": 0,  # 0 is default and the most deterministic
    "max_tokens": 2048,  # 2048 works for most models, but may have to be reduced for some models
    "num_retries": 3,  # Number of times to retry a question if it fails
}

multiple_choice_questions = False 

if multiple_choice_questions is False:
    # Auto evaluation is only supported for open-ended questions and involves an LLM evaluating results against a set of criteria
    # Criteria can be found in `auto_eval.py` create_eval_prompt function
    auto_eval_rounds = (
        10  # Number of rounds of auto evaluation to run to then average the scores
    )
    auto_eval_model = ("gpt-4o", "custom")
    auto_eval_hyperparams = {
        "temperature": 0,
        "max_tokens": 2048,
        "batch_size": 30,
    }

hotz_reflection = False
benchmark_name = 'Benchmark' if not multiple_choice_questions else 'Multi-Benchmark'
# date_now = '2024-07-20' #datetime.now().strftime('%Y-%m-%d') #'2024-07-20'
date_now = datetime.now().strftime('%Y-%m-%d') #'2024-07-20'
folder_name = f"{date_now}-{benchmark_name}" #"2024-06-21-Multi-Benchmark (temp=0)" 

answers_save_path = f"./{folder_name}/llm_outputs"
answers_hotz_save_path = f"./{folder_name}/hotz_reflection"
auto_eval_save_path = f"./{folder_name}/auto_eval_outputs"
auto_eval_hotz_save_path = f"./{folder_name}/auto_eval_hotz_outputs"
stats_save_path = f"./{folder_name}/tables_and_charts"

execution_steps = [
    "get_llm_answers",
    # "hotz_reflection",
    "evaluate_answers",
    "generate_statistics", 
]

In [32]:
# from llm_service import custom_llm_service
# custom_service = custom_llm_service()
# response = custom_service.completion(
#     messages=[{"role": "user", "content": "10 + 10 ="}],
#     model="gpt-4-turbo-preview",
#     max_tokens=100,
#     temperature=0,
# )
# print(response)

## Run Benchmark

In [36]:
def calc_incomplete_llm_answers(all_llm_answers, auto_eval_save_path, sub_eval_folders, date_now):
    all_llm_evals = load_all_llm_answers_from_json(auto_eval_save_path,
        prefix_replace='auto_eval-', sub_folders=sub_eval_folders)
    skip_evals = set(all_llm_evals.keys() & set(all_llm_answers.keys()))
    print(f'Skipping existing LLM evals (in {date_now} folder):', skip_evals)
    incomplete_llm_answers = {model: value for model, value in all_llm_answers.items() 
                               if model_clean(model) not in skip_evals}
    return incomplete_llm_answers

In [40]:
# Load in benchmark questions
if multiple_choice_questions:
    benchmark_questions = json.load(
        open("linguistic_benchmark_multi_choice.json", "r", encoding="utf-8")
    )
    sub_eval_folders = [""]

    def answer_validation_func(x):
        return validation_func(
            x, json_key="ANSWER", list_of_values=["A", "B", "C", "D"]
        )

else:
    benchmark_questions = json.load(
        open("linguistic_benchmark.json", "r", encoding="utf-8")
    )
    sub_eval_folders = [f"/round_{r+1}" for r in range(auto_eval_rounds)]
sub_answer_folders = [f"/round_{r+1}" for r in range(answer_rounds)]

if "get_llm_answers" in execution_steps:
    print("1. GETTING LLM ANSWERS")
    # Load in any existing answers and evals to avoid overwriting them
    for n in range(answer_rounds):
        print(f"\n----- Round: {n+1} of {answer_rounds} -----")
        answer_save_path_round = f"{answers_save_path}/round_{n+1}"
        all_llm_answers = load_all_llm_answers_from_json(
            answer_save_path_round, prefix_replace="final_answers-"
        )
        print(
            f"Skipping existing LLM answers (in {answer_save_path_round} folder):",
            list(all_llm_answers.keys()),
        )
        answer_models_run = [
            model
            for model in answer_models
            if model_clean(model[0]) not in all_llm_answers.keys()
        ]
        questions = benchmark_questions.copy()
        if multiple_choice_questions:
            questions = {}
            for model, llm_service in answer_models_run:
                clean_model = model_clean(model)
                questions[clean_model] = benchmark_questions.copy()
                for q in questions[clean_model]:
                    prompt, correct_letter = construct_multiple_choice_question(q)
                    q.update(
                        {
                            "multi_choice_question": prompt,
                            "correct_letter": correct_letter,
                        }
                    )
        all_llm_answers = await get_llm_answers(
            questions,
            answer_models_run,
            answer_hyperparams,
            answer_save_path_round,
            multiple_choice_questions,
            validation_func=(
                answer_validation_func if multiple_choice_questions else lambda x: True
            ),
        )
    print("-- DONE ANSWERS --\n")


if "hotz_reflection" in execution_steps:
    print("1.5 GETTING HOTZ REFLECTION ANSWERS")
    # Load in any existing answers and evals to avoid overwriting them
    for n in range(answer_rounds):
        print(f"\n----- Round: {n+1} of {answer_rounds} -----")
        answer_save_path_round = f"{answers_save_path}/round_{n+1}"
        answer_hotz_save_path_round = f"{answers_hotz_save_path}/round_{n+1}"
        all_llm_answers = load_all_llm_answers_from_json(
            answer_save_path_round, prefix_replace="final_answers-"
        )
        all_hotz_llm_answers = load_all_llm_answers_from_json(
            answer_hotz_save_path_round, prefix_replace="final_answers-"
        )
        print(
            f"Skipping existing LLM answers (in {answer_hotz_save_path_round} folder):",
            list(all_hotz_llm_answers.keys()),
        )
        answer_models_run = [
            model
            for model in answer_models
            if model_clean(model[0]) not in all_hotz_llm_answers.keys()
        ]
        all_llm_questions = {
            model: info.to_dict("records") for model, info in all_llm_answers.items()
        }
        if multiple_choice_questions:
            for model, questions in all_llm_questions.items():
                for q in questions:
                    prompt = construct_hotz_reflection_question(q)
                    q.update({"multi_choice_question_pre": q["multi_choice_question"]})
                    q.update({"model_answer_pre": q["model_answer"]})
                    q.update({"multi_choice_question": prompt})

        all_llm_answers = await get_llm_answers(
            all_llm_questions,
            answer_models_run,
            answer_hyperparams,
            answer_hotz_save_path_round,
            multiple_choice_questions,
            validation_func=(
                answer_validation_func if multiple_choice_questions else lambda x: True
            ),
        )
    print("-- DONE ANSWERS --\n")


if "evaluate_answers" in execution_steps:
    print("2. EVALUATING ANSWERS")
    all_llm_answers = load_all_llm_answers_from_json(
        answers_save_path,
        prefix_replace="final_answers-",
        sub_folders=sub_answer_folders,
    )
    if multiple_choice_questions:
        incomplete_llm_answers = all_llm_answers
        all_llm_evals = score_multiple_choice_answers(
            incomplete_llm_answers, auto_eval_save_path
        )
        if hotz_reflection:
            print("\nEvaluate Hotz Reflections")
            all_llm_answers = load_all_llm_answers_from_json(
                answers_hotz_save_path,
                prefix_replace="final_answers-",
                sub_folders=sub_answer_folders,
            )
            incomplete_llm_answers = calc_incomplete_llm_answers(
                all_llm_answers, auto_eval_hotz_save_path, sub_eval_folders, date_now
            )
            all_llm_evals = score_multiple_choice_answers(
                incomplete_llm_answers, auto_eval_hotz_save_path
            )
    else:
        all_llm_eval_messages = create_all_llm_eval_messages(
            all_llm_answers, benchmark_questions
        )
        for n in range(auto_eval_rounds):
            print(f"- Round: {n+1} of {auto_eval_rounds} -")
            incomplete_llm_answers = calc_incomplete_llm_answers(
                all_llm_eval_messages, auto_eval_save_path, [f"/round_{n+1}"], date_now
            )
            all_llm_eval_responses = await get_llm_eval_responses(
                incomplete_llm_answers,
                model_info=auto_eval_model,
                hyperparams=auto_eval_hyperparams,
                validation_func=lambda x: validation_func(
                    x, json_key="score", list_of_values=[0, 20, 40, 60, 80, 100]
                ),
            )
            all_llm_scores = extract_all_scores(all_llm_eval_responses)
            auto_eval_save_path_n = f"{auto_eval_save_path}/round_{n+1}"
            all_auto_results = create_auto_eval_json(
                all_llm_scores,
                all_llm_eval_responses,
                all_llm_answers,
                benchmark_questions,
                auto_eval_save_path_n,
            )
    print("-- DONE EVAL --\n")


if "generate_statistics" in execution_steps:
    print("3. GENERATING STATISTICS")
    all_stats_dfs = {}
    save_info = [
        {
            "path": auto_eval_save_path,
            "chart_title": "LLM Linguistic Benchmark Performance",
            "type": "",
        }
    ]
    if hotz_reflection:
        save_info.append(
            {
                "path": auto_eval_hotz_save_path,
                "chart_title": "LLM Linguistic Benchmark Performance (Hotz Reflection)",
                "type": "-Hotz",
            }
        )
    for info in save_info:
        save_path = info["path"]
        chart_title = info["chart_title"]
        info_type = info["type"]
        print("Eval for path:", save_path)
        all_llm_evals = load_all_llm_answers_from_json(
            save_path,
            prefix_replace="auto_eval-",
            sub_folders=sub_eval_folders,
        )
        print("All LLM Evaluations:", all_llm_evals)

        stats_df = get_llm_stats(
            all_llm_evals, stats_save_path, file_suffix=info_type, bootstrap_n=10000
        )
        if multiple_choice_questions:
            for model, evals_df in all_llm_evals.items():
                # evals_df['invalid_answer_letter'] = evals_df.apply(lambda x: x['json_answer_letter'] not in ['A', 'B', 'C', 'D'], axis=1)
                incorrect_letter_count = evals_df["invalid_answer_letter"].sum()
                print(model, incorrect_letter_count)
                stats_df.loc[model, "invalid_outputs"] = incorrect_letter_count

        display(stats_df)
        barplot, plt = create_performance_chart(
            stats_df.reset_index(),
            chart_title,
            highlight_models=["o1-preview"],
        )
        barplot.figure.savefig(f"{stats_save_path}/performance_chart{info_type}.png")
        plt.show()
        all_stats_dfs[chart_title] = stats_df
    print("-- DONE STATS --\n")

16:46:47 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:46:47 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:46:47 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:46:47 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:46:47 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


1. GETTING LLM ANSWERS

----- Round: 1 of 1 -----
Skipping existing LLM answers (in ./2024-10-30-Benchmark/llm_outputs/round_1 folder): []
Running  Benchmark for gpt-4-turbo-preview
> Processing batch 1-5 ex 20


Request to litellm:
litellm.completion(messages=[{'role': 'user', 'content': 'You have a weighted graph where multiple paths to the same node have identical cumulative costs. However, one of the edges in a tie leads to a goal state. Will UCS always find the optimal solution? Explain why or why not.'}], model='gpt-4-turbo-preview', temperature=0, max_tokens=2048, num_retries=3)




Request to litellm:
litellm.completion(messages=[{'role': 'user', 'content': 'What happens if UCS is run on a graph where all edge costs are the same? How does its behavior compare to BFS?'}], model='gpt-4-turbo-preview', temperature=0, max_tokens=2048, num_retries=3)


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperatu

16:48:40 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:48:40 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:48:40 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:48:40 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:48:40 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


> Processing batch 6-10 ex 20


Request to litellm:
litellm.completion(messages=[{'role': 'user', 'content': 'How does DFS perform on a cyclic graph without a visited-node tracker? Describe a scenario where it gets stuck in an infinite loop.'}], model='gpt-4-turbo-preview', temperature=0, max_tokens=2048, num_retries=3)


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0, 'max_tokens': 2048, 'max_retries': 3, 'extra_body': {}}


POST Request Sent from LiteLLM:
curl -X POST \
https://api.openai.com/v1/ \
-d '{'model': 'gpt-4-turbo-preview', 'messages': [{'role': 'user', 'content': 'How does DFS perform on a cyclic graph without a visited-node tracker? Describe a scenario where it gets stuck in an infinite loop.'}], 'temperature': 0, 'max_tokens': 2048, 'extra_body': {}}'




Request to litellm:
litellm.completion(messages=[{'role': 'user', 'content': 'In a building with 10 floors, DFS is used to sea

16:50:38 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:50:38 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:50:38 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:50:38 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:50:38 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


> Processing batch 11-15 ex 20


Request to litellm:
litellm.completion(messages=[{'role': 'user', 'content': 'How does BFS behave when applied to an unweighted graph compared to a weighted graph where all edge weights are equal?'}], model='gpt-4-turbo-preview', temperature=0, max_tokens=2048, num_retries=3)




Request to litellm:
litellm.completion(messages=[{'role': 'user', 'content': 'An A* search algorithm is given a heuristic that always underestimates the true cost. Is A* guaranteed to find the optimal solution? Why or why not?'}], model='gpt-4-turbo-preview', temperature=0, max_tokens=2048, num_retries=3)


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0, 'max_tokens': 2048, 'max_retries': 3, 'extra_body': {}}


POST Request Sent from LiteLLM:
curl -X POST \
https://api.openai.com/v1/ \
-d '{'model': 'gpt-4-turbo-preview', 'messages': [{'role': 'user', 'content': 'How does BFS behave when

16:52:33 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:52:33 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:52:33 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:52:33 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
16:52:33 - LiteLLM:WARNING: utils.py:346 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


> Processing batch 16-20 ex 20


Request to litellm:
litellm.completion(messages=[{'role': 'user', 'content': 'You have a search problem with a graph containing cycles. Which search algorithm—DFS, BFS, or UCS—is most likely to get stuck, and why?'}], model='gpt-4-turbo-preview', temperature=0, max_tokens=2048, num_retries=3)


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False


Request to litellm:
litellm.completion(messages=[{'role': 'user', 'content': 'A delivery robot is exploring a city grid using BFS. Some streets are closed, and the robot is instructed to find the shortest path while avoiding these blocks. How does BFS behave differently from UCS in this scenario?'}], model='gpt-4-turbo-preview', temperature=0, max_tokens=2048, num_retries=3)


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'temperature': 0, 'max_tokens': 2048, 'max_retries': 3, 'extra_body': {}}


PO

IndexError: tuple index out of range

In [ ]:
raise

## Inspect Answers for a given model

In [ ]:
# model = 'gpt-4-turbo-preview'
# answers = all_llm_evals[model][['question', 'model_answer', 'json_answer', 'correct_letter', 'score']]
# for row in answers.to_dict('records'):
#     print('--------------')
#     print('\nQuestion\n', row['question'])
#     print('\nModel Answer\n', row['model_answer'])
#     print('\nJson Answer\n', row['json_answer'])
#     print('\nCorrect Letter\n', row['correct_letter'])    
#     print('\nScore\n', row['score'])

## Inspect Hotz Reflections Performance

In [ ]:
all_llm_evals = load_all_llm_answers_from_json(
    auto_eval_hotz_save_path, 
    prefix_replace='auto_eval-',
    sub_folders=sub_eval_folders,
)

answer_change_stats = {}
for model, model_evals in all_llm_evals.items():
    model_evals['json_answer_letter_pre'] = model_evals['model_answer_pre'].map(extract_valid_json).map(lambda x: None if x is None else x['ANSWER'])

    compare_letters = model_evals[['json_answer_letter', 'json_answer_letter_pre', 'correct_letter']].copy()
    compare_letters['changed_answer'] = compare_letters['json_answer_letter'] != compare_letters['json_answer_letter_pre']
    compare_letters['changed_answer_perc'] = (compare_letters['changed_answer'] / len(compare_letters)) * 100
    compare_letters['correct'] = compare_letters['json_answer_letter'] == compare_letters['correct_letter']
    compare_letters['correct_pre'] = compare_letters['json_answer_letter_pre'] == compare_letters['correct_letter']

    compare_letters[['correct', 'correct_pre', 'changed_answer_perc']].sum()
    answer_change_stats[model] = compare_letters[['correct_pre', 'correct', 'changed_answer_perc']].sum()
final_answer_change_df = pd.DataFrame(answer_change_stats).transpose().sort_values('correct', ascending=False)
final_answer_change_df.rename(columns={
    'correct_pre': 'Correct (pre-reflection)', 
    'correct': 'Correct (post-reflection)', 
    }, inplace=True)
final_answer_change_df

In [ ]:
hotz_ref = 'LLM Liguisitc Benchmark Performance (Hotz Reflection)'
ref = 'LLM Liguisitc Benchmark Performance'

all_stats_dfs[hotz_ref]['mean_score_pre'] = all_stats_dfs[ref]['mean_score']

all_stats_dfs[hotz_ref]['diff'] = all_stats_dfs[hotz_ref]['mean_score'] - all_stats_dfs[hotz_ref]['mean_score_pre']
all_stats_dfs[hotz_ref]
diff_from_pre = all_stats_dfs[hotz_ref][['mean_score', 'mean_score_pre', 'diff']]
diff_from_pre

In [ ]:
diff_from_pre_final_raw = diff_from_pre.merge(final_answer_change_df[['changed_answer_perc']], left_index=True, right_index=True)
diff_from_pre_final = diff_from_pre_final_raw[['mean_score_pre', 'mean_score', 'diff', 'changed_answer_perc']].round(0)
diff_from_pre_final.rename(columns={
    "mean_score_pre": "Pre-Reflection Score",
    "mean_score": "Post-Reflection Score",
    "diff": "Difference",
    "changed_answer_perc": "Answers Changed (%)",
}, inplace=True)
#diff_from_pre_final = diff_from_pre_final.astype(int).astype(str) + '%'
diff_from_pre_final

In [ ]:
diff_from_pre_final['Answers Changed (%)'].mean()

# Inspect Auto Eval Consistancy

In [ ]:
all_llm_evals = load_all_llm_answers_from_json(
    auto_eval_save_path, 
    prefix_replace='auto_eval-',
    sub_folders=sub_eval_folders,
)
models = list(all_llm_evals.keys())


model = models[3]
print(f"Model: {model}")
auto_eval_agg = all_llm_evals[model].set_index('level_0').groupby('index').agg({'score': ['mean', 'min', 'max']})
auto_eval_agg.index.name = 'Question #'
auto_eval_agg

In [ ]:
# tables_folder = "2024-06-12-Multi-Benchmark (temp=1)/tables_and_charts"
# stats_df = pd.read_csv(f"{tables_folder}/final_stats.csv", index_col=0)
# barplot, plt = create_performance_chart(stats_df.reset_index())
# barplot.figure.savefig(f"{tables_folder}/performance_chart.png")